In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from pandasql import sqldf

import matplotlib.pyplot as plt
import seaborn as sns

# **Задача**

По поведению пользователей на площадке rabota.ru предсказать, на какие вакансии пользователь в дальнейшем откликнется или позвонит.

В рамках задачи вам будет необходимо разработать модель, которая будет опираться на взаимодействия пользователей (соискателей) с вакансией. На выходе модель должна отдавать список из n рекомендованных вакансий, отсортированных по релевантности.


# **Данные**

## ***train_mfti.parquet*** – сырые данные, которые можно использовать для обучения модели
* event_date – дата взаимодейтсвия
* event_timestamp – timestamp взаимодействия в секундах 
* vacancy_id_ - id вакансии, с которой было взаимодействие
* cookie_id – id пользователя по его браузеру/ip/устройству
* user_id – id пользователя на сайте rabota.ru (есть только для зарегистрированных пользователей)
* event_type – тип взаимодействия


In [2]:
df = pd.read_parquet('train_mfti.parquet')
display(df.head(3))
print(df.shape)

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy


(12292588, 6)


## ***test_public_mfti.parquet*** – часть теста, с открытым таргетом, для проверки работоспособности решений.
* cookie_id - id пользователя по его браузеру/ip/устройству
* vacancy_id_ - список вакансий, на которые пользователь откликнулся или позвонил в течение месяца после окончания данных train


In [3]:
df1_ = pd.read_parquet('test_public_mfti.parquet')
display(df1_.head(3))
print(df1_.shape)

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."


(772, 2)


## ***test_private_users_mfti.parquet*** – часть теста, с закрытым таргетом для итоговой проверки решений
* cookie_id - id пользователя по его браузеру/ip/устройству


In [4]:
df2_ = pd.read_parquet('test_private_users_mfti.parquet')
display(df2_.head(3))
print(df2_.shape)

,cookie_id
0,0018914ba3e54011b28fa715583d3354
1,0035c298d8c64f368ae730a9cca9bb20
2,00956458877448ec9fba87fb97443fdf


(3086, 1)


## ***test_private_sample_submission_mfti.parquet*** – файл с примером предсказаний, который требуется получить по итогу хакатона 
* cookie_id - id пользователя по его браузеру/ip/устройству
* predictions – список из 5 id вакансий, которые модель предсказала как наиболее релеватные для данного пользователя


In [5]:
df3_ = pd.read_parquet('test_private_sample_submission_mfti.parquet')
display(df3_.head(3))
print(df3_.shape)

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[100100, 100101, 100102, 100103, 100104]"
1,0035c298d8c64f368ae730a9cca9bb20,"[100100, 100101, 100102, 100103, 100104]"
2,00956458877448ec9fba87fb97443fdf,"[100100, 100101, 100102, 100103, 100104]"


(3086, 2)


# **Метрика**
* Результаты модели будут оцениваться по метрике ***precision@5***. 
* При демонстрации своего решения по завершению хакатона, вам необходимо будет предоставить ***код для обучения и инференса модели, презентацию по проделанной работе и файл с предсказаниями*** для пользователей из списка test_private_users_mfti.parquet в формате файла test_private_sample_submission_mfti.parquet


# **Рекомендации по решению**
* В таких задачах, чаще всего, baseline моделью считается рекомендация самых популярных вакансий для всех пользователей. Попробуйте написать полный pipeline с использованием этой простой модели и отталкиваться от него в процессе ваших исследований
* Обратите внимания на то, что если пользователь каким-либо образом взаимодействовал с вакансией в обучающих данных, то для этого пользователя данной вакансии не будет в списке меток теста. Мы не хотим рекомендовать пользователю то, что он уже видел и с чем взаимодействовал.
* Постарайтесь не использовать файл test_public_mfti.parquet для постоянной валидации своих моделей, это может привести к переобучению. Вместо этого лучше подготовить свой валидационный датасет.


# **Решение** (baseline)

In [236]:
# cookie_id - список vacancy_id (с которыми было взаимодействие)
df_ = df.groupby(by='cookie_id')['vacancy_id_'].unique()

In [6]:
# Смотрим, какие типы взаимодействий имеются
df['event_type'].unique()

array(['show_vacancy', 'preview_click_vacancy', 'click_contacts',
       'preview_click_contacts', 'click_favorite',
       'preview_click_favorite', 'preview_click_phone',
       'preview_click_response', 'click_phone', 'click_response'],
      dtype=object)

In [7]:
# Выполним mapping для типов взаимодействий:
df['rating'] = df['event_type'].map({'show_vacancy': 0,
                                     'preview_click_vacancy': 1,
                                     'click_favorite': 2,
                                     'preview_click_favorite': 3,
                                     'click_response': 4,
                                     'preview_click_response': 5,
                                     'click_contacts': 6,
                                     'preview_click_contacts': 7,
                                     'click_phone': 8,
                                     'preview_click_phone': 9})

In [8]:
# Количество значений в каждом столбце и тип данных
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12292588 entries, 0 to 12292587
Data columns (total 7 columns):
 #   Column           Non-Null Count     Dtype 
---  ------           --------------     ----- 
 0   event_date       12292588 non-null  object
 1   event_timestamp  12292588 non-null  int64 
 2   vacancy_id_      12292588 non-null  int64 
 3   cookie_id        12292588 non-null  object
 4   user_id          8711177 non-null   object
 5   event_type       12292588 non-null  object
 6   rating           12292588 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 656.5+ MB


In [9]:
# Вывод количества уникальных id
print(f'{"Столбец:" : <15} {"Количество уникальных:"}')
for col_id in ('vacancy_id_', 'cookie_id', 'user_id'):
    print(f'{col_id.split(sep="_")[0] : <15} {df[col_id].nunique(dropna=False)}')

Столбец:        Количество уникальных:
vacancy         160167
cookie          330180
user            209336


In [10]:
df['cookie_id'].value_counts()[:10]

924398d361a0454c8c30845c2b4c5747    4898
7d8cc5a0fef44378a2d90a237cda288e    3343
5934d5b8a0b348829d8efabe69c733eb    3315
b7dd2f20fdd6472ab62f8d86a739cd5d    3312
353b56c4fa6447d0ba0f08b42d86e51f    2795
6b3281f474314f96b75a7d1a06f09eed    2478
43cef1cac8f646a1adf620cb023fb311    2262
57013f363bb64956a65c4061c75e43e1    2143
54c29201d1eb4a9bb446090e631e948a    2120
784a284e0bd34228903d95cff1559ca4    2103
Name: cookie_id, dtype: int64

In [11]:
# Интересный момент, выглядит будто "накрутка"
df[df['cookie_id'] == '57013f363bb64956a65c4061c75e43e1']['vacancy_id_'].value_counts()

164246    1199
201312     788
219158     149
207169       3
257747       2
177710       2
Name: vacancy_id_, dtype: int64

In [12]:
# ТОП-5 cookie по количеству вакансий, с которыми они взаиимодействовали
df.groupby(by='cookie_id').nunique()['vacancy_id_'].sort_values(ascending=False)[:5]

cookie_id
924398d361a0454c8c30845c2b4c5747    4541
784a284e0bd34228903d95cff1559ca4    1228
728c61d1db294cb59e6032a65706b964    1106
6b3281f474314f96b75a7d1a06f09eed     961
7d8cc5a0fef44378a2d90a237cda288e     826
Name: vacancy_id_, dtype: int64

## Оценки вакансий

Создадим новую таблицу:
1. vacancy_id - идентификатор вакансии
2. reach - охват [количество уникальных cookie, взаимодействующих как-либо с вакансией]
3. interactions - [количевство взаимодействий]
4. sum_total_score - [сумма всех баллов]
5. avg_total_score - среднее от всех баллов [sum_total_score / interactions]
6. ERR - доля вовлеченности по охвату [sum_total_score / reach]
7. uinteractions - [количевство уникальных взаимодействий]
8. sum_max_score - сумма максимальных баллов [сумма максимальных баллов от каждого cookie]
9. avg_max_score - среднее от максимального балла каждого пользователя [sum_max_score / reach]
10. sum_total_uscore - [сумма уникальных баллов от каждого пользователя]
11. avg_total_uscore - среднее от уникальных баллов каждого пользователя [sum_total_uscore / interactions]
12. uERR - доля уникальной вовлеченности по охвату [sum_total_uscore / reach]


**балл* - вес типа взаимодействия пользователя с вакансией [0-9]

In [12]:
query = """
    WITH dd AS (
        SELECT
            vacancy_id_ AS vacancy_id,
            COUNT(DISTINCT cookie_id) AS reach,
            COUNT(rating) AS interactions,
            SUM(rating) AS sum_total_score,
            AVG(rating) AS avg_total_score
        FROM 
            df
        GROUP BY 
            vacancy_id_
    )
    SELECT
        d.vacancy_id,
        d.reach,
        d.interactions,
        d.sum_total_score,
        d.avg_total_score,
        CAST(d.sum_total_score AS float) / d.reach AS ERR,
        SUM(sc.uinteractions) AS uinteractions,
        SUM(sc.max_rating) AS sum_max_score,
        CAST(SUM(sc.max_rating) AS float) / d.reach AS avg_max_score,
        SUM(sc.uscore) AS sum_total_uscore,
        CAST(SUM(sc.uscore) AS float) / SUM(sc.uinteractions) AS avg_total_uscore,
        AVG(sc.uscore) AS uERR
    FROM dd AS d
    LEFT JOIN (
        SELECT
            vacancy_id_,
            cookie_id,
            COUNT(DISTINCT rating) AS uinteractions,
            MAX(rating) AS max_rating,
            SUM(DISTINCT rating) AS uscore
        FROM 
            df
        GROUP BY 
            vacancy_id_, cookie_id
    ) AS sc
    ON d.vacancy_id = sc.vacancy_id_
    GROUP BY 
        d.vacancy_id
        """

data = sqldf(query)

In [13]:
data

,vacancy_id,reach,interactions,sum_total_score,avg_total_score,ERR,uinteractions,sum_max_score,avg_max_score,sum_total_uscore,avg_total_uscore,uERR
0,100001,20,49,50,1.020408,2.500000,43,39,1.950000,49,1.139535,2.450000
1,100002,224,847,866,1.022432,3.866071,497,458,2.044643,609,1.225352,2.718750
2,100003,29,60,40,0.666667,1.379310,54,35,1.206897,37,0.685185,1.275862
3,100004,3,4,1,0.250000,0.333333,4,1,0.333333,1,0.250000,0.333333
4,100005,3,9,10,1.111111,3.333333,6,9,3.000000,10,1.666667,3.333333
...,...,...,...,...,...,...,...,...,...,...,...,...
160162,260163,9,18,9,0.500000,1.000000,17,8,0.888889,8,0.470588,0.888889
160163,260164,1,1,5,5.000000,5.000000,1,5,5.000000,5,5.000000,5.000000
160164,260165,42,102,74,0.725490,1.761905,87,61,1.452381,71,0.816092,1.690476
160165,260166,7,13,12,0.923077,1.714286,11,9,1.285714,11,1.000000,1.571429


In [149]:
# Копия, чтобы быстро откатиться назад
v_metrics = data.copy()

In [150]:
# Ввожу новую метрику оценки
v_metrics['my_score'] = np.log(v_metrics['interactions']) * (v_metrics['avg_max_score'])

In [151]:
# Ввожу новую метрику оценки 2
v_metrics['my_score2'] = np.log(v_metrics['reach']) * (v_metrics['avg_max_score'])

In [152]:
# Ввожу новую метрику оценки 3
v_metrics['my_score3'] = v_metrics['reach'] * (v_metrics['avg_max_score'])

In [153]:
v_metrics.describe()

,vacancy_id,reach,interactions,sum_total_score,avg_total_score,ERR,uinteractions,sum_max_score,avg_max_score,sum_total_uscore,avg_total_uscore,uERR,my_score,my_score2,my_score3
count,160167.00000,160167.000000,160167.000000,160167.000000,160167.000000,160167.000000,160167.000000,160167.000000,160167.000000,160167.000000,160167.000000,160167.000000,160167.000000,160167.000000,160167.000000
mean,180084.00000,29.210668,76.748569,69.059257,1.418143,2.924368,58.509206,47.392472,2.051753,57.414830,1.486024,2.462626,5.162810,3.698908,47.392472
std,46236.37462,175.746648,484.159623,410.200334,1.466716,3.244840,364.011392,285.754177,1.392563,344.377954,1.440155,2.113878,3.347428,2.596214,285.754177
min,100001.00000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,140042.50000,4.000000,10.000000,8.000000,0.612245,1.448276,8.000000,6.000000,1.200000,7.000000,0.687500,1.303571,2.708050,1.732868,6.000000
50%,180084.00000,10.000000,24.000000,20.000000,0.866667,2.142857,19.000000,15.000000,1.640000,18.000000,0.971223,1.923077,4.969813,3.532677,15.000000
75%,220125.50000,23.000000,59.000000,54.000000,1.333333,3.285714,46.000000,38.000000,2.263158,46.000000,1.444444,2.969697,7.278045,5.364159,38.000000
max,260167.00000,18662.000000,59105.000000,44962.000000,9.000000,197.000000,38090.000000,28130.000000,9.000000,34529.000000,9.000000,42.000000,29.555835,25.749141,28130.000000


In [154]:
# Похоже на попытку "поднять" вакансию в ТОП
v_metrics[(v_metrics['ERR'] > 100)]

,vacancy_id,reach,interactions,sum_total_score,avg_total_score,ERR,uinteractions,sum_max_score,avg_max_score,sum_total_uscore,avg_total_uscore,uERR,my_score,my_score2,my_score3
45597,145598,1,22,138,6.272727,138.0,3,8,8.0,18,6.000000,18.0,24.728340,0.0,8.0
63009,163010,1,36,106,2.944444,106.0,5,7,7.0,18,3.600000,18.0,25.084633,0.0,7.0
80907,180908,1,31,197,6.354839,197.0,5,8,8.0,28,5.600000,28.0,27.471898,0.0,8.0
126336,226337,1,16,103,6.437500,103.0,6,9,9.0,36,6.000000,36.0,24.953299,0.0,9.0
129448,229449,1,18,103,5.722222,103.0,6,8,8.0,32,5.333333,32.0,23.122974,0.0,8.0


In [155]:
# Выбирать рекомендацию по оценке "my_score" не совсем правильно,
# т.к. много записей с высоким баллом, но с малым количеством пользователей
v_metrics[v_metrics['my_score'] > 25]

,vacancy_id,reach,interactions,sum_total_score,avg_total_score,ERR,uinteractions,sum_max_score,avg_max_score,sum_total_uscore,avg_total_uscore,uERR,my_score,my_score2,my_score3
1395,101396,823,4297,7005,1.630207,8.511543,2095,2731,3.318348,4360,2.081146,5.297691,27.760208,22.275921,2731.0
16822,116823,10901,40023,41019,1.024886,3.762866,26479,27167,2.492157,34529,1.304015,3.167508,26.409907,23.168608,27167.0
63009,163010,1,36,106,2.944444,106.000000,5,7,7.000000,18,3.600000,18.000000,25.084633,0.000000,7.0
74952,174953,6237,21864,24577,1.124085,3.940516,15763,16554,2.654161,21576,1.368775,3.459355,26.521957,23.192731,16554.0
80381,180382,3340,12650,13430,1.061660,4.020958,8456,9077,2.717665,11646,1.377247,3.486826,25.669464,22.050387,9077.0
80907,180908,1,31,197,6.354839,197.000000,5,8,8.000000,28,5.600000,28.000000,27.471898,0.000000,8.0
82869,182870,7871,29649,39097,1.318662,4.967221,20505,22592,2.870283,32478,1.583906,4.126286,29.555835,25.749141,22592.0
84066,184067,80,468,1374,2.935897,17.175000,207,375,4.687500,653,3.154589,8.162500,28.820945,20.540750,375.0
135107,235108,4,65,126,1.938462,31.500000,15,24,6.000000,54,3.600000,13.500000,25.046324,8.317766,24.0
145951,245952,700,3490,5295,1.517192,7.564286,1746,2197,3.138571,3350,1.918671,4.785714,25.603389,20.561034,2197.0


In [156]:
# По-моему, метрика "my_score2" хорошо отражает весомые взаимодействия
# и количество пользователей
v_metrics[v_metrics['my_score2'] > 20]

,vacancy_id,reach,interactions,sum_total_score,avg_total_score,ERR,uinteractions,sum_max_score,avg_max_score,sum_total_uscore,avg_total_uscore,uERR,my_score,my_score2,my_score3
1395,101396,823,4297,7005,1.630207,8.511543,2095,2731,3.318348,4360,2.081146,5.297691,27.760208,22.275921,2731.0
16822,116823,10901,40023,41019,1.024886,3.762866,26479,27167,2.492157,34529,1.304015,3.167508,26.409907,23.168608,27167.0
74952,174953,6237,21864,24577,1.124085,3.940516,15763,16554,2.654161,21576,1.368775,3.459355,26.521957,23.192731,16554.0
80381,180382,3340,12650,13430,1.061660,4.020958,8456,9077,2.717665,11646,1.377247,3.486826,25.669464,22.050387,9077.0
82869,182870,7871,29649,39097,1.318662,4.967221,20505,22592,2.870283,32478,1.583906,4.126286,29.555835,25.749141,22592.0
84066,184067,80,468,1374,2.935897,17.175000,207,375,4.687500,653,3.154589,8.162500,28.820945,20.540750,375.0
93330,193331,3437,10820,12146,1.122551,3.533896,7896,8444,2.456794,10518,1.332067,3.060227,22.821529,20.004085,8444.0
107422,207423,11988,38607,38884,1.007175,3.243577,28115,25981,2.167251,32902,1.170265,2.744578,22.888743,20.354084,25981.0
145951,245952,700,3490,5295,1.517192,7.564286,1746,2197,3.138571,3350,1.918671,4.785714,25.603389,20.561034,2197.0


In [157]:
# Ранжирую вакансии по новой оценке (по охвату, если одинаковый балл и т.д.)
# method='first' для того, чтобы у каждой вакансии было уникальное значение в рейтинге
v_metrics['my_score_rank'] = v_metrics[['my_score', 'reach', 'interactions', 'sum_total_score', 'sum_total_uscore', 'sum_max_score']].apply(tuple,axis=1)\
                                    .rank(method='first',ascending=False).astype(int)

In [158]:
# Ранжирую вакансии по новой оценке (по охвату, если одинаковый балл и т.д.)
v_metrics['my_score2_rank'] = v_metrics[['my_score2', 'reach', 'interactions', 'sum_total_score', 'sum_total_uscore', 'sum_max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [159]:
# Ранжирую вакансии по новой оценке (по охвату, если одинаковый балл и т.д.)
v_metrics['my_score3_rank'] = v_metrics[['my_score3', 'reach', 'interactions', 'sum_total_score','sum_total_uscore', 'sum_max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [160]:
v_metrics['reach_rank'] = v_metrics[['reach', 'interactions', 'sum_total_score', 'uinteractions', 'sum_total_uscore', 'sum_max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [161]:
v_metrics['interactions_rank'] = v_metrics[['interactions', 'reach', 'sum_total_score', 'uinteractions', 'sum_total_uscore', 'sum_max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [162]:
v_metrics['sum_total_score_rank'] = v_metrics[['sum_total_score', 'reach', 'interactions', 'uinteractions', 'sum_total_uscore', 'sum_max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [163]:
v_metrics['avg_total_score_rank'] = v_metrics[['avg_total_score', 'reach', 'interactions', 'sum_total_score', 'sum_total_uscore', 'sum_max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [164]:
v_metrics['ERR_rank'] = v_metrics[['ERR', 'reach', 'interactions', 'uERR', 'sum_total_score', 'uinteractions', 'sum_total_uscore', 'sum_max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [165]:
v_metrics['uinteractions_rank'] = v_metrics[['uinteractions', 'reach', 'interactions', 'sum_total_score', 'sum_total_uscore', 'sum_max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [166]:
v_metrics['sum_max_score_rank'] = v_metrics[['sum_max_score', 'reach', 'interactions', 'sum_total_score', 'avg_max_score', 'sum_total_uscore']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [167]:
v_metrics['avg_max_score_rank'] = v_metrics[['avg_max_score', 'reach', 'interactions', 'sum_total_score', 'sum_max_score', 'sum_total_uscore']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [168]:
v_metrics['sum_total_uscore_rank'] = v_metrics[['sum_total_uscore', 'reach', 'interactions', 'sum_total_score', 'avg_total_uscore', 'sum_max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [169]:
v_metrics['avg_total_uscore_rank'] = v_metrics[['avg_total_uscore', 'reach', 'interactions', 'sum_total_score', 'sum_total_uscore', 'avg_max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [170]:
v_metrics['uERR_rank'] = v_metrics[['uERR', 'reach', 'interactions', 'sum_total_score', 'ERR', 'uinteractions', 'sum_total_uscore', 'sum_max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [171]:
# Сортировка по среднему значению всех рангов
v_metrics['ultra_rank'] = v_metrics.loc[:, 'my_score_rank':'uERR_rank'].mean(axis=1).rank(method='first', ascending=True).astype(int)

In [172]:
# Функция для вычисления логарифма и среднего значения
def log_mean(row):
    return np.log(row).mean()

# Сортировка по среднему значению логарифмов всех рангов
v_metrics['ultra_log_rank'] = v_metrics.loc[:, 'my_score_rank':'uERR_rank'].apply(log_mean, axis=1).rank(method='first', ascending=True).astype(int)

In [173]:
v_metrics.sort_values('reach_rank').head(5)

,vacancy_id,reach,interactions,sum_total_score,avg_total_score,ERR,uinteractions,sum_max_score,avg_max_score,sum_total_uscore,avg_total_uscore,uERR,my_score,my_score2,my_score3,my_score_rank,my_score2_rank,my_score3_rank,reach_rank,interactions_rank,sum_total_score_rank,avg_total_score_rank,ERR_rank,uinteractions_rank,sum_max_score_rank,avg_max_score_rank,sum_total_uscore_rank,avg_total_uscore_rank,uERR_rank,ultra_rank,ultra_log_rank
160153,260154,18662,59105,44962,0.760714,2.409281,38090,28130,1.507341,34472,0.905014,1.847176,16.561264,14.823561,28130.0,457,122,1,1,1,1,96460,68624,1,1,91438,2,88737,84375,10865,3
98113,198114,16674,50187,38400,0.765138,2.302987,35317,27654,1.658510,32223,0.912393,1.932530,17.950905,16.123383,27654.0,225,62,2,2,2,5,95638,73059,2,2,78991,5,87611,79697,9769,5
103403,203404,16284,45634,32158,0.704694,1.974822,34046,23533,1.445161,26419,0.775980,1.622390,15.504276,14.015081,23533.0,786,179,6,3,3,7,105892,91744,3,6,98102,7,108176,99378,16951,10
11504,111505,15088,35095,23346,0.665223,1.547322,29712,19375,1.284133,20911,0.703790,1.385936,13.439498,12.355486,19375.0,2184,466,10,4,9,14,112888,114710,5,10,113742,12,118213,115208,23578,20
102607,202608,14524,41620,32483,0.780466,2.236505,31459,24083,1.658152,28016,0.890556,1.928945,17.636662,15.890996,24083.0,264,70,5,5,4,6,93119,75931,4,5,79002,6,90778,79791,10052,7


In [174]:
1/0
v_metrics.to_parquet('vacancy_metrics.parquet')

In [175]:
1/0
v_metrics = pd.read_parquet('vacancy_metrics.parquet')

In [177]:
def precision_n(predictions, df_test_list_top, k=5):
    
    # Количество рекомендаций по k вакансий:
    n_rec_k = len(df_test_list_top)*k
    
    # Подсчет валидных вакансий:
    matches = 0
    for row in range(len(predictions)):
        row_match = set(predictions['top_recommendations'][row]).intersection(set(df_test_list_top['vacancy_id_'][row]))  
        matches += len(row_match)
    
    try: 
        precisions = matches / n_rec_k
    except:
        precisions = 0

    return precisions

In [240]:
def top_vacancy(df1=df1_, df2=v_metrics, df3=df_, rank='my_score2_rank', num=5):
    
    """
    df1 >>> датафрейм, из которого нужно получить список "cookie_id" для рекомендаций
    df2 >>> датафрейм, с рангами
    df3 >>> датафрейм (лог-файл), здесь идет поиск вакансий, с которыми 
    пользователь уже взаимодействовал 
    rank >>> название столбца df2 с рейтингом вакансий
    num >>> кол-во вакансий для рекомендации
    """
    
    # Выбираю список cookie, для которых рекомендуются вакансии
    test_list = list(df1['cookie_id'])

    # Новый датафрейм с сookie_id и рекомендуемыми вакансиями
    result = pd.DataFrame(columns=['cookie_id', 'top_recommendations'])

    # Список вакансий по рангу
    candidates = df2.sort_values(rank)['vacancy_id'].tolist()
    
    # Для каждого пользователя свои рекомендации
    for cookie in test_list:
        
        # Список с вакансиями, с которыми пользователь уже взаимодействовал
        not_recommend = df3[cookie].tolist()
        # Список из num рекомендуемых вакансий
        recommend = []
        for vacancy in candidates:
            if vacancy in not_recommend:
                continue
            recommend.append(vacancy)
            if len(recommend) == num:
                break
        # Запись результатов
        result = pd.concat([result, pd.DataFrame({'cookie_id': cookie,
                                                  'top_recommendations': [recommend]})], ignore_index=True)
        
    return precision_n(result, df1, num)

In [241]:
for col in v_metrics.loc[:, 'my_score_rank':'ultra_log_rank'].columns:
    print(f'precision@5: {top_vacancy(df1_, v_metrics, df_, col, 5): <23} по "{col}"')

precision@5: 0.007512953367875648    по "my_score_rank"
precision@5: 0.008031088082901554    по "my_score2_rank"
precision@5: 0.01917098445595855     по "my_score3_rank"
precision@5: 0.02072538860103627     по "reach_rank"
precision@5: 0.019689119170984457    по "interactions_rank"
precision@5: 0.021502590673575128    по "sum_total_score_rank"
precision@5: 0.0                     по "avg_total_score_rank"
precision@5: 0.0                     по "ERR_rank"
precision@5: 0.021502590673575128    по "uinteractions_rank"
precision@5: 0.01917098445595855     по "sum_max_score_rank"
precision@5: 0.0                     по "avg_max_score_rank"
precision@5: 0.021502590673575128    по "sum_total_uscore_rank"
precision@5: 0.0                     по "avg_total_uscore_rank"
precision@5: 0.0                     по "uERR_rank"
precision@5: 0.0010362694300518134   по "ultra_rank"
precision@5: 0.020207253886010364    по "ultra_log_rank"


# Вывод:

Наилучший результат по ***baseline*** дает ранжирование по:

[1.] сумме всех баллов

[1.] количеству уникальных взаимодействий (от каждого польхователя их может быть <= 10)

[1.] сумме уникальных баллов

[2.] охвату

[3.] выдуманной метрике (ultra_log_rank) [ранг по среднему значению логарифмов других рангов]

*Вероятно, можно улучшить полученные результаты по precision@5, если заняться подбором баллов за каждый тип взаимодействия*